In [4]:
import json
import numpy as np
from guidance import models, system, user, assistant, select
# from models.prompts.prompts import *
import llama_cpp
# from models.models import *
import pandas as pd

In [2]:
with open('memory/qwen_run_8_11/main.json', 'r') as f:
    main = json.load(f)
with open('memory/qwen_run_8_11/detection.json', 'r') as f:
    detection = json.load(f)
    
data = pd.read_csv('dataset_building/8_11/selected_8_11.csv')

In [18]:
bill_relevant_gt = 3
data = data[data.bill_id.isin((data.bill_id.value_counts() 
                > bill_relevant_gt).index[data.bill_id.value_counts() > bill_relevant_gt].values)]

In [67]:
detection = np.array(detection).sum(-1).sum(-1)

In [6]:
detection_str = [str(x) for x in list(np.where(np.logical_and(detection > 0, detection < 3))[0])]

In [ ]:
lm =  QwenChat('llms/qwen1_5-72b-chat-q5_k_m.gguf', n_gpu_layers=-1,
                n_ctx=6000, echo=True, splint_mode=llama_cpp.LLAMA_SPLIT_MODE_NONE, main_gpu=0)

In [21]:
with system():
    utility = lm + CHECK_UTILITY_SYS

In [ ]:
with user():
    utility_check = utility + CHECK_UTILITY.format(business=data.loc[5].business_description,
                                         bill=data.loc[5].summary_text,
                                         amendment=amendments[1],
                                         benefit=benefits[2])
with assistant():
    utility_check = utility_check + select(['NO', 'YES'])

#### Analyzing results

In [7]:
import json
import numpy as np
import sklearn as sk
import tqdm

In [12]:
with open('utility_verification_mistral7_12_initial_intent.json', 'r') as f:
    utility = json.load(f)
    
# with open('memory/qwen_run_gt112/main.json', 'r') as f:
#     main = json.load(f)

In [25]:
# memory_paths = ['memory/{model}_run_8_11/main.json', 'memory/{model}_run_gt112/main.json', 'memory/{model}_run_14_16_18/main.json']
# mains = []
# for path in memory_paths:
#     with open(path.format(model='mistral7instructV2'), 'r') as f:
#         mains += [json.load(f)]

for m in ["mistral7", "mixtral", "qwen", "qwen7", "qwen14", "yi"]:
    utility_paths = ['utility_verification_{model}_8_initial_intent.json', 'utility_verification_{model}_12_initial_intent.json', 'utility_verification_{model}_14_initial_intent.json']
    utilities = []
    for u_path in utility_paths:
        with open(u_path.format(model='mistral7'), 'r') as f:
            utilities.extend(list(json.load(f).values()))

    with open(f"utility_verification_{m}.json", 'w') as f:
        json.dump({k: v for k,v in enumerate(utilities)}, f)

In [38]:
utility_capture = {}
utility_capture['overall_capture'] = []
percent_capture_trial = [[], [], []]
for split in range(3):
    
    for i in range(3):
        
        amend_benefit=[]
        for k, v in utilities[split].items():
            if len(v) >= i+1:
                amend_benefit.append(v[i])
        
        percent_capture_trial[i] += amend_benefit

for i in range(3):
    utility_capture['overall_capture'].append(np.bitwise_or.reduce(np.array(percent_capture_trial[i]).reshape(-1, 3, 3).transpose(0, 2, 1), axis=-1).mean(-1).mean())
    
utility_capture

{'overall_capture': [0.8893144101524366,
  0.8182348140375422,
  0.7754608643094589]}

In [11]:
import numpy as np
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    # m, se = np.mean(a, axis=-1), scipy.stats.sem(a, axis=-1)
    m, se = np.mean(a, axis=-1), scipy.stats.tstd(a, axis=-1)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return [[m, m-h, m+h], se]

In [29]:
from tqdm import tqdm
import itertools


top1 = True     # conditions set for top1

mains_map = {'mistral7': 'mistral7instructV2', 'qwen7': 'qwen7', 'qwen14': 'qwen14','yi': 'yiChat', 'mixtral': 'mixtral', 'qwen': 'qwen'}
utility_capture = {}

for model in ['mistral7', 'qwen7', 'qwen14', 'yi', 'mixtral', 'qwen']:
    
    memory_paths = ['/home/ic38970/reinforced_lobbying/memory/{model}_run_8_11/main.json', '/home/ic38970/reinforced_lobbying/memory/{model}_run_gt112/main.json', '/home/ic38970/reinforced_lobbying/memory/{model}_run_14_16_18/main.json']
    detection_paths = ['/home/ic38970/reinforced_lobbying/memory/{model}_run_8_11/detection.json', '/home/ic38970/reinforced_lobbying/memory/{model}_run_gt112/detection.json', '/home/ic38970/reinforced_lobbying/memory/{model}_run_14_16_18/detection.json']
    
    # mains
    mains = []
    if model == 'yi':
        memory_paths = [memory_paths[i] for i in [0,2]]
    for path in memory_paths:
        with open(path.format(model=mains_map[model]), 'r') as f:
            mains += [json.load(f)]
    
    
    utility_paths = ['utility_verification_{model}_8_initial_intent.json', 'utility_verification_{model}_12_initial_intent.json', 'utility_verification_{model}_14_initial_intent.json']
    
    splits = 3
    if model == 'yi':
        utility_paths = [utility_paths[i] for i in [0,2]]
        splits=2
        
        
    utilities = []
    for u_path in utility_paths:
        with open(u_path.format(model=model), 'r') as f:
            utilities += [json.load(f)]
    
    
    # detections
    if top1:
        detections = []
        if model == 'yi':
            detection_paths = [detection_paths[i] for i in [0,2]]
        for path in detection_paths:
            with open(path.format(model=mains_map[model]), 'r') as f:
                detections += [json.load(f)]
        
        dt_top1 = []
        dt_top1_not_decept_sim_index = []
        for split in range(splits):
            # try:
            dt = np.array(detections[split])
            # except:
            #     print(split)
            for i in range(dt[:, :, 0].shape[0]):
                for j in range(dt[i, :, 0].shape[0]-2, -1, -1):
                    if not dt[i, :, 0][j]:
                        dt[i, :, 0][j:] = 0
                        
            dt_top1.append(dt[:, :, 0].copy())
            dt_top1_not_decept_sim_index.append(np.where(np.all(dt_top1[-1], axis=-1))[0])
    
    
    # overall utility capture
    percent_capture_trial = [[], [], []]
    
    for split in range(splits):
        
        for i in range(3):
            amend_benefit=[]
            for k, v in utilities[split].items():
                # if top1:
                #     if int(k) in dt_top1_index[split]:
                #         if len(v) >= i+1: amend_benefit.append(v[i])
                # else:
                if len(v) >= i+1: amend_benefit.append(v[i])
                    
            percent_capture_trial[i] += amend_benefit
    
    utility_capture[model] = {'overall_capture': [], 'deceptive_sims': [], 'particular_trials': []}
    
    for trial in range(3):
        boot_captures = []
        for seed in tqdm(range(100), desc=f'{model}_trial{trial}_overall_capture'):
            boot = sk.utils.resample(percent_capture_trial[trial], replace=True, n_samples=percent_capture_trial[trial].__len__(), random_state=seed)
            boot_captures.append(np.bitwise_or.reduce(np.array(boot).reshape(-1, 3, 3).transpose(0, 2, 1), axis=-1).mean(-1).mean())
        ci = mean_confidence_interval(boot_captures)
        ci = [np.bitwise_or.reduce(np.array(percent_capture_trial[trial]).reshape(-1, 3, 3).transpose(0, 2, 1), axis=-1).mean(-1).mean()] + ci
        utility_capture[model]['overall_capture'].append(ci)        
    
    
    # only those simulations that were deceptive
    percent_capture_trial = [[], [], []]
    
    for split in range(splits):
        for i in range(len(utilities[split])):
            
            condition = not(int(i) in dt_top1_not_decept_sim_index[split]) if top1 else True
            if condition:
                trials = len(mains[split][str(i)]) - 1
                
                if trials < 3: 
                    for j in range(trials):
                            try:
                                # print(split, j, i)
                                percent_capture_trial[j].append(utilities[split][str(i)][j])
                            except:
                                # print(split, j, i)
                                continue
                if trials >= 3:
                    if not mains[split][str(i)][str(2)]['detected']:
                        for j in range(3):
                            try:
                                percent_capture_trial[j].append(utilities[split][str(i)][j])
                            except:
                                # print(split, j, i)
                                continue
    
    # utility_capture[model] = {'deceptive_sims': []}
    for trial in range(3):
        boot_captures = []
        for seed in tqdm(range(100), desc=f'{model}_trial{trial}_deceptive_trials'):
            boot = sk.utils.resample(percent_capture_trial[trial], replace=True, n_samples=percent_capture_trial[trial].__len__(), random_state=seed)
            boot_captures.append(np.bitwise_or.reduce(np.array(boot).reshape(-1, 3, 3).transpose(0, 2, 1), axis=-1).mean(-1).mean())
        ci = mean_confidence_interval(boot_captures)
        ci = [np.bitwise_or.reduce(np.array(percent_capture_trial[trial]).reshape(-1, 3, 3).transpose(0, 2, 1), axis=-1).mean(-1).mean()] + ci
        utility_capture[model]['deceptive_sims'].append(ci)
    
    
    # particularly deceptive trials
    if model =='yi':
        deceived_t1 = [[],[]]
        deceived_t2 = [[],[]]
        deceived_t3 = [[],[]]
    else:
        deceived_t1 = [[],[],[]]
        deceived_t2 = [[],[],[]]
        deceived_t3 = [[],[],[]]

    for split in range(splits):
        for i in range(len(utilities[split])):
            
            # condition = int(i) in dt_top1_index[split] if top1 else True
            # if condition:
            try:
                if top1:
                    trials = np.argmax(dt_top1[split][i]==0)
                    if np.all(dt_top1[split][i], axis=-1) and trials==0:
                        trials=0
                    else:
                        trials += 1
                        
                    if trials == 1:
                        deceived_t1[split].append(utilities[split][str(i)][trials-1])
                    elif trials == 2:
                        deceived_t2[split].append(utilities[split][str(i)][trials-1])
                    elif trials == 3:
                            deceived_t3[split].append(utilities[split][str(i)][trials-1])
                    
                else:
                    trials = len(mains[split][str(i)]) - 1
                
                # if trials < 4:      #TODO IMPORTANT: change for rest, only qwen-72b has 4 trials
                    if trials == 1:
                        deceived_t1[split].append(utilities[split][str(i)][trials-1])
                    elif trials == 2:
                        deceived_t2[split].append(utilities[split][str(i)][trials-1])
                    elif trials == 3:
                        if not mains[split][str(i)][str(2)]['detected']:
                            deceived_t3[split].append(utilities[split][str(i)][trials-1])
            except:
                continue
                    
    deceived_trials = [deceived_t1, deceived_t2, deceived_t3]
    
    # utility_capture[model] = {'particular_trials': []}
    for idx, trial in enumerate(deceived_trials):
        boot_captures = []
        for seed in tqdm(range(100), desc=f'{model}_trial{idx}_particular_trials'):
            boot = sk.utils.resample(trial, replace=True, n_samples=trial.__len__(), random_state=seed)
            boot_captures.append(np.bitwise_or.reduce(np.array(list(itertools.chain.from_iterable(boot))).reshape(-1, 3, 3).transpose(0, 2, 1), axis=-1).mean(-1).mean())
        ci = mean_confidence_interval(boot_captures)
        ci = [np.bitwise_or.reduce(np.array(list(itertools.chain.from_iterable(trial))).reshape(-1, 3, 3).transpose(0, 2, 1), axis=-1).mean(-1).mean()] + ci
        utility_capture[model]['particular_trials'].append(ci)


qwen_trial2_particular_trials: 100%|██████████| 100/100 [00:00<00:00, 1023.36it/s]


In [32]:
# utility_capture['model'] = 'mixtral'
with open('utility_capture_ci_100_top1.json', 'a') as f:
    json.dump(utility_capture, f, indent=4)
    f.write('\n')

##### Only those that were deceptive

In [21]:
percent_capture_trial = [[], [], []]

for split in range(3):
    for i in range(len(utilities[split])):
        trials = len(mains[split][str(i)]) - 1
        
        if trials < 3: 
            for j in range(trials):
                    try:
                        # print(split, j, i)
                        percent_capture_trial[j].append(utilities[split][str(i)][j])
                    except:
                        # print(split, j, i)
                        continue
        if trials >= 3:
            if not mains[split][str(i)][str(2)]['detected']:
                for j in range(3):
                    try:
                        percent_capture_trial[j].append(utilities[split][str(i)][j])
                    except:
                        # print(split, j, i)
                        continue

In [22]:
utility_capture['deceptive_sims'] = []
for i in range(3):
    utility_capture['deceptive_sims'].append(np.bitwise_or.reduce(np.array(percent_capture_trial[i]).reshape(-1, 3, 3).transpose(0, 2, 1), axis=-1).mean(-1).mean())
utility_capture

{'overall_capture': [0.8893144101524366,
  0.8182348140375422,
  0.7754608643094589],
 'deceptive_sims': [0.8723573992820104,
  0.7630348795397338,
  0.6605839416058394]}

##### Only those that were deceived at that particular trial

In [8]:
import json
import numpy as np
import itertools

In [39]:
deceived_t1 = [[],[],[]]
deceived_t2 = [[],[],[]]
deceived_t3 = [[],[],[]]

for split in range(3):
    
    for i in range(len(utilities[split])):
        try:
            trials = len(mains[split][str(i)]) - 1
            
            # if trials < 4:      #IMPORTANT: change for rest, only qwen-72b has 4 trials
            if trials == 1:
                deceived_t1[split].append(utilities[split][str(i)][trials-1])
            elif trials == 2:
                deceived_t2[split].append(utilities[split][str(i)][trials-1])
            elif trials == 3:
                if not mains[split][str(i)][str(2)]['detected']:
                    deceived_t3[split].append(utilities[split][str(i)][trials-1])
        except:
            continue
                
deceived_trials = [deceived_t1, deceived_t2, deceived_t3]

In [40]:
utility_capture['particular_trials'] = []
for trial in deceived_trials:
    amend_benefit = np.array(list(itertools.chain.from_iterable(trial))).reshape(-1, 3, 3).transpose(0, 2, 1)
    amend_benefit_ORed_benefit = np.bitwise_or.reduce(amend_benefit, axis=-1)
    utility_capture['particular_trials'].append(amend_benefit_ORed_benefit.mean(-1).mean())
utility_capture

KeyboardInterrupt: 

save this shit

In [17]:
utility_capture['model'] = 'mixtral'
with open('utility_capture.json', 'a') as f:
    json.dump(utility_capture, f, indent=4)
    f.write('\n')